In [2]:
import torch, torchvision
from PIL import Image

print("Torch:", torch.__version__)
print("TorchVision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Pillow OK")


Torch: 2.3.1
TorchVision: 0.18.1
CUDA available: False
Pillow OK


In [8]:
# ============================================================
# 🔹 CPU-ONLY Single Image Inference (EfficientNet-B2)
# ============================================================

import os
import time
import json
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import efficientnet_b2
from PIL import Image

# ---------------- HARD CPU LOCK ----------------
os.environ["CUDA_VISIBLE_DEVICES"] = ""
torch.set_grad_enabled(False)

device = torch.device("cpu")
torch.set_num_threads(max(1, os.cpu_count() // 2))

# ---------------- Paths ----------------
BASE_DIR = r"D:\mymodel\selected_models"
MODEL_PATH = os.path.join(BASE_DIR, "best_model_smart_combo_v2.pth")
CLASSES_PATH = os.path.join(BASE_DIR, "classes.json")

IMAGE_PATH = r"D:\ISIC-images\ISIC_0000032.jpg"

# ---------------- Params ----------------
resolution = 260
top_k = 3
dropout_decoder = 0.3   # 🔑 MUST MATCH TRAINING

# ---------------- Resize + Pad ----------------
class ResizeAndPad:
    def __init__(self, output_size, fill=0):
        self.output_size = output_size
        self.fill = fill

    def __call__(self, img):
        w, h = img.size
        ratio = min(self.output_size / w, self.output_size / h)
        new_w, new_h = int(w * ratio), int(h * ratio)
        img = transforms.functional.resize(img, (new_h, new_w), antialias=True)
        canvas = Image.new("RGB", (self.output_size, self.output_size), self.fill)
        canvas.paste(
            img,
            ((self.output_size - new_w) // 2,
             (self.output_size - new_h) // 2)
        )
        return canvas

# ---------------- Transforms ----------------
infer_tf = transforms.Compose([
    ResizeAndPad(resolution),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ---------------- Model ----------------
class SimpleClassifier(nn.Module):
    def __init__(self, encoder, num_classes, dropout=0.3):
        super().__init__()
        self.encoder = encoder
        with torch.no_grad():
            dummy = torch.zeros(1, 3, resolution, resolution)
            out_features = self.encoder(dummy).shape[1]

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(dropout),          # 🔑 REQUIRED
            nn.Linear(out_features, num_classes)
        )

    def forward(self, x):
        return self.head(self.encoder(x))

# ---------------- Load Classes ----------------
with open(CLASSES_PATH, "r") as f:
    classes = json.load(f)

num_classes = len(classes)
print(f"📦 Classes ({num_classes}): {classes}")

# ---------------- Load Model ----------------
backbone = efficientnet_b2(weights=None)
encoder = nn.Sequential(*backbone.features)
model = SimpleClassifier(encoder, num_classes, dropout=dropout_decoder)

model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
model.eval()

print("✅ Model loaded successfully (CPU)")

# ---------------- Inference ----------------
img = Image.open(IMAGE_PATH).convert("RGB")
x = infer_tf(img).unsqueeze(0)

start = time.perf_counter()
logits = model(x)
latency_ms = (time.perf_counter() - start) * 1000

probs = torch.softmax(logits, dim=1)[0]
conf, pred = torch.max(probs, dim=0)
topk_conf, topk_idx = torch.topk(probs, top_k)

# ---------------- Output ----------------
print("\n🖼️ Inference Result")
print(f"Image      : {os.path.basename(IMAGE_PATH)}")
print(f"Prediction : {classes[pred.item()]}")
print(f"Confidence : {conf.item():.4f}")
print(f"Latency    : {latency_ms:.2f} ms")

print("\n🔝 Top-K:")
for i in range(top_k):
    print(f" {i+1}. {classes[topk_idx[i].item()]} ({topk_conf[i].item():.4f})")


📦 Classes (8): ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'scc', 'vasc']
✅ Model loaded successfully (CPU)

🖼️ Inference Result
Image      : ISIC_0000032.jpg
Prediction : nv
Confidence : 0.8777
Latency    : 35.25 ms

🔝 Top-K:
 1. nv (0.8777)
 2. mel (0.0576)
 3. akiec (0.0163)
